In [1]:
import os
import sys

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_dir = os.path.dirname(notebook_dir)

if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import pandas as pd
df_a = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\training_data.csv")
mapping = pd.read_csv(r"C:\Users\johna\anaconda3\envs\twitter-analytics-env\twitter_issues_dashboard\twitter_issues_dashboard\data\02_intermediate\topic_mapping.csv")

df = df_a.merge(right=mapping, how='inner', right_on='topic_sortedc_str', left_on='topic_sortedc_str')[['cleaned_text', 'topic_hl']]
df['topic_hl_encoded'] = df['topic_hl'].astype("category").cat.codes
df

,cleaned_text,topic_hl,topic_hl_encoded
0,#royalmail ridiculous rule not being able to s...,Customer Service,0
1,#RoyalMail and it's 50 year delivery,Customer Service,0
2,So #royalmail not delivering despite same addr...,Customer Service,0
3,Ask #royalmail if you've paid enough postage...,Customer Service,0
4,#royalmail this how my post arrived today #emp...,Customer Service,0
...,...,...,...
2618,MARSHALL WACE LLP declares that on June 09 it ...,Finance News,1
2619,MARSHALL WACE LLP declares that on June 13 it ...,Finance News,1
2620,MARSHALL WACE LLP declares that on June 15 it ...,Finance News,1
2621,MARSHALL WACE LLP declares that on June 21 it ...,Finance News,1


In [3]:
from machine_learning.preprocess_ml import DataPipeline

C:\Users\johna\anaconda3\envs\twitter-analytics-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pipeline = DataPipeline(df, target_col='topic_hl_encoded', model='cardiffnlp/twitter-xlm-roberta-base')
train_loader = pipeline.get_train_data(batch_size=16)
val_loader = pipeline.get_val_data(batch_size=16)
test_loader = pipeline.get_test_data(batch_size=16)

In [5]:
from machine_learning.train_models import TransformerFineTuner

In [6]:
output_dir = r"C:\Users\johna\OneDrive\Desktop\models_twitter_dash\output"
logging_dir = r"C:\Users\johna\OneDrive\Desktop\models_twitter_dash\logging"

transformer_fine_tuner = TransformerFineTuner(train_loader, val_loader, 'cardiffnlp/twitter-roberta-base', 3, output_dir, logging_dir)
transformer_fine_tuner.train()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`